# NFL Competition

# Feature Engineering e Modelo de Machine Learning

- Version: 1.0: usando padrao do fork: https://www.kaggle.com/bestpredict/location-eda-8eb410
        *    Resultado: 0.012744
        *    LB: 0.01363
   
- Version: 2.0: adicionado Feature Selection com LOFO Importance
        *    Resultado: 0.012780
        *    LB: 0.01365

- Version: 3.0: adicionado novas features (apenas feature fxx + old_data + YardLine_std)
        *    Resultado: 0.012614
        *    LB: 0.01361
        
- Version: 4.0: adicionado novas features (turf)
        *    Resultado: 0.012624
        *    LB: 0.01361
        
- Version: 5.0: adicionado novas features (game_time)
        *    Resultado: 0.012635
        *    LB: 0.01362
        
- Version: 6.0: adicionado novas features (feat1, feat2, feat3, feat4) e removido (Turf + game_time)
        *    Resultado: 0.012536
        *    LB: Não é permitido
        
- Version: 7.0: alteração do modelo de bagging
        *    Resultado: 0.012474
        *    LB: 0.01362

- Version: 8.0: adicionado novas features (accY, est_prev_yards)
        *    Resultado: 0.012616
        *    LB: 0.01362
        
- Version: 9.0: adicionado novas features (norm_quat,mod_quat,norm_X,norm_Y,norm_A,norm_S)
    
        *    Resultado: 0.012604
        *    LB: 0.01361

- Version: 12.0: realizando stacking dos modelos NN e RF
    
        *    Resultado: 0.012616
        *    LB: 

## 1. Importa os pacotes e o dataset de treino

In [1]:
# Importar os principais pacotes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
import re
import codecs
import time
import datetime
import tsfresh
import pandasql as ps
import gc

# Evitar que aparece os warnings
import warnings
warnings.filterwarnings("ignore")

# Seta algumas opções no Jupyter para exibição dos datasets
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Variavel para controlar o treinamento no Kaggle
TRAIN_OFFLINE = True

/Users/rodrigolima82/anaconda3/lib/python3.7/site-packages/statsmodels/compat/pandas.py:23: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
# Importa os pacotes de algoritmos
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Importa os pacotes de algoritmos de redes neurais (Keras)
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.utils import to_categorical
from keras.layers import Dense,Input,Flatten,concatenate,Dropout,Lambda,BatchNormalization
from keras.layers import Activation
from keras.models import Sequential, Model
from keras.callbacks import Callback,EarlyStopping,ModelCheckpoint
import keras.backend as K
from keras.optimizers import Adam, Nadam, RMSprop
from keras import optimizers
#from keras_lookahead import Lookahead
#from keras_radam import RAdam

# Importa pacotes do sklearn
from sklearn import preprocessing
import sklearn.metrics as mtr
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import scale, MinMaxScaler, StandardScaler
from sklearn import model_selection

Using TensorFlow backend.


In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [4]:
if TRAIN_OFFLINE:
    train  = pd.read_csv('../data/train.csv', dtype={'WindSpeed': 'object'})
    new_ft = pd.read_csv('../data/nfl-sample-features.csv')
else:
    train  = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', dtype={'WindSpeed': 'object'})

#train = train[:2200]

In [5]:
outcomes = train[['GameId','PlayId','Yards']].drop_duplicates()

## 2. Feature Engineering

In [6]:
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans

def strtofloat(x):
    try:
        return float(x)
    except:
        return -1
    
def get_time(x):
    x = x.split(":")
    return int(x[0])*60 + int(x[1])

def map_weather(txt):
    ans = 1
    if pd.isna(txt):
        return 0
    if 'partly' in txt:
        ans*=0.5
    if 'climate controlled' in txt or 'indoor' in txt:
        return ans*3
    if 'sunny' in txt or 'sun' in txt:
        return ans*2
    if 'clear' in txt:
        return ans
    if 'cloudy' in txt:
        return -ans
    if 'rain' in txt or 'rainy' in txt:
        return -2*ans
    if 'snow' in txt:
        return -3*ans
    return 0

def OffensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0, 'QB' : 0, 'RB' : 0, 'TE' : 0, 'WR' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def DefensePersonnelSplit(x):
    dic = {'DB' : 0, 'DL' : 0, 'LB' : 0, 'OL' : 0}
    for xx in x.split(","):
        xxs = xx.split(" ")
        dic[xxs[-1]] = int(xxs[-2])
    return dic

def orientation_to_cat(x):
    x = np.clip(x, 0, 360 - 1)
    try:
        return str(int(x/15))
    except:
        return "nan"    
    
def uid_aggregation(comb, main_columns, uids, aggregations):
    X = pd.DataFrame()
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_'+agg_type
                temp_df = comb[[col, main_column]]
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                X[new_col_name] = comb[col].map(temp_df)
                del temp_df
                gc.collect()
    return X

def transform_time_quarter(str1):
    return int(str1[:2])*60 + int(str1[3:5])

def transform_time_all(str1,quarter):
    if quarter<=4:
        return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    if quarter ==5:
        return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60

In [20]:
def create_features_01(df, deploy=False):
    def new_X(x_coordinate, play_direction):
        if play_direction == 'left':
            return 120.0 - x_coordinate
        else:
            return x_coordinate

    def new_line(rush_team, field_position, yardline):
        if rush_team == field_position:
            # offense starting at X = 0 plus the 10 yard endzone plus the line of scrimmage
            return 10.0 + yardline
        else:
            # half the field plus the yards between midfield and the line of scrimmage
            return 60.0 + (50 - yardline)

    def new_orientation(angle, play_direction):
        if play_direction == 'left':
            new_angle = 360.0 - angle
            if new_angle == 360.0:
                new_angle = 0.0
            return new_angle
        else:
            return angle

    def euclidean_distance(x1,y1,x2,y2):
        x_diff = (x1-x2)**2
        y_diff = (y1-y2)**2
        return np.sqrt(x_diff + y_diff)

    def back_direction(orientation):
        if orientation > 180.0:
            return 1
        else:
            return 0

    def velocity(x2, x1, sec) :
        return (x2 - x1) / sec
    
    def diff_x(b, c, theta) :
        if 90.0 < theta < 270.0 :
            return np.sqrt(((b ** 2) + (c ** 2)) - 2 * b * c * np.cos(theta))
        else :
            return 0
        
    def diff_y(b, c, theta) :
        if theta <= 90.0 and theta >= 270.0 :
            return - np.sqrt(((b ** 2) + (c ** 2)) - 2 * b * c * np.cos(theta))
        else :
            return 0
        
    def stop_period(speed, acc) :
        return speed / acc   

    def new_roll_velocity(x1, y1, x2, y2) :  
        x_diff = np.sqrt((x1 - x2) ** 2)
        y_diff = np.sqrt((y1 - y2) ** 2)
        return np.sqrt(x_diff + y_diff) 
    
    def update_yardline(df):
        new_yardline = df[df['NflId'] == df['NflIdRusher']]
        new_yardline['YardLine'] = new_yardline[['PossessionTeam','FieldPosition','YardLine']].apply(lambda x: new_line(x[0],x[1],x[2]), axis=1)
        new_yardline = new_yardline[['GameId','PlayId','YardLine']]
        return new_yardline

    def update_orientation(df, yardline):
        df['X'] = df[['X','PlayDirection']].apply(lambda x: new_X(x[0],x[1]), axis=1)
        df['Orientation'] = df[['Orientation','PlayDirection']].apply(lambda x: new_orientation(x[0],x[1]), axis=1)
        df['Dir'] = df[['Dir','PlayDirection']].apply(lambda x: new_orientation(x[0],x[1]), axis=1)
        df = df.drop('YardLine', axis=1)
        df = pd.merge(df, yardline, on=['GameId','PlayId'], how='inner')
        return df

    def back_features(df):
        carriers = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','NflIdRusher','X','Y','Orientation','Dir','YardLine']]
        carriers['back_from_scrimmage'] = carriers['YardLine'] - carriers['X']
        carriers['back_oriented_down_field'] = carriers['Orientation'].apply(lambda x: back_direction(x))
        carriers['back_moving_down_field'] = carriers['Dir'].apply(lambda x: back_direction(x))
        carriers = carriers.rename(columns={'X':'back_X','Y':'back_Y'})
        carriers = carriers[['GameId','PlayId','NflIdRusher','back_X','back_Y','back_from_scrimmage','back_oriented_down_field','back_moving_down_field']]
        return carriers

    def features_relative_to_back(df, carriers):
        player_distance = df[['GameId','PlayId','NflId','X','Y']]
        player_distance = pd.merge(player_distance, carriers, on=['GameId','PlayId'], how='inner')
        player_distance = player_distance[player_distance['NflId'] != player_distance['NflIdRusher']]
        player_distance['dist_to_back'] = player_distance[['X','Y','back_X','back_Y']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)

        player_distance = player_distance.groupby(['GameId','PlayId','back_from_scrimmage','back_oriented_down_field','back_moving_down_field'])\
                                         .agg({'dist_to_back':['min','max','mean','std']})\
                                         .reset_index()
        player_distance.columns = ['GameId','PlayId','back_from_scrimmage','back_oriented_down_field','back_moving_down_field',
                                   'min_dist','max_dist','mean_dist','std_dist']

        return player_distance

    def defense_features(df):
        rusher = df[df['NflId'] == df['NflIdRusher']][['GameId','PlayId','Team','X','Y']]
        rusher.columns = ['GameId','PlayId','RusherTeam','RusherX','RusherY']

        defense = pd.merge(df,rusher,on=['GameId','PlayId'],how='inner')
        defense = defense[defense['Team'] != defense['RusherTeam']][['GameId','PlayId','X','Y','RusherX','RusherY']]
        defense['def_dist_to_back'] = defense[['X','Y','RusherX','RusherY']].apply(lambda x: euclidean_distance(x[0],x[1],x[2],x[3]), axis=1)

        defense = defense.groupby(['GameId','PlayId'])\
                         .agg({'def_dist_to_back':['min','max','mean','std']})\
                         .reset_index()
        defense.columns = ['GameId','PlayId','def_min_dist','def_max_dist','def_mean_dist','def_std_dist']

        return defense
            
    
    def static_features(df):
        
        
        add_new_feas = []

        ## Height
        df['PlayerHeight_dense'] = df['PlayerHeight'].apply(lambda x: 12*int(x.split('-')[0])+int(x.split('-')[1]))
        
        add_new_feas.append('PlayerHeight_dense')

        ## Time
        df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
        df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))

        df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
        df['PlayerBirthDate'] =df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

        ## Age
        seconds_in_year = 60*60*24*365.25
        df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
        add_new_feas.append('PlayerAge')

        ## WindSpeed
        df['WindSpeed_ob'] = df['WindSpeed'].apply(lambda x: x.lower().replace('mph', '').strip() if not pd.isna(x) else x)
        df['WindSpeed_ob'] = df['WindSpeed_ob'].apply(lambda x: (int(x.split('-')[0])+int(x.split('-')[1]))/2 if not pd.isna(x) and '-' in x else x)
        df['WindSpeed_ob'] = df['WindSpeed_ob'].apply(lambda x: (int(x.split()[0])+int(x.split()[-1]))/2 if not pd.isna(x) and type(x)!=float and 'gusts up to' in x else x)
        df['WindSpeed_dense'] = df['WindSpeed_ob'].apply(strtofloat)
        add_new_feas.append('WindSpeed_dense')

        ## Weather
        df['GameWeather_process'] = df['GameWeather'].str.lower()
        df['GameWeather_process'] = df['GameWeather_process'].apply(lambda x: "indoor" if not pd.isna(x) and "indoor" in x else x)
        df['GameWeather_process'] = df['GameWeather_process'].apply(lambda x: x.replace('coudy', 'cloudy').replace('clouidy', 'cloudy').replace('party', 'partly') if not pd.isna(x) else x)
        df['GameWeather_process'] = df['GameWeather_process'].apply(lambda x: x.replace('clear and sunny', 'sunny and clear') if not pd.isna(x) else x)
        df['GameWeather_process'] = df['GameWeather_process'].apply(lambda x: x.replace('skies', '').replace("mostly", "").strip() if not pd.isna(x) else x)
        df['GameWeather_dense'] = df['GameWeather_process'].apply(map_weather)
        add_new_feas.append('GameWeather_dense')

        ## Orientation and Dir
        df["Orientation_ob"] = df["Orientation"].apply(lambda x : orientation_to_cat(x)).astype("object")
        df["Dir_ob"] = df["Dir"].apply(lambda x : orientation_to_cat(x)).astype("object")

        df["Orientation_sin"] = df["Orientation"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
        df["Orientation_cos"] = df["Orientation"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
        
        df["Dir_sin"] = df["Dir"].apply(lambda x : np.sin(x/360 * 2 * np.pi))
        df["Dir_cos"] = df["Dir"].apply(lambda x : np.cos(x/360 * 2 * np.pi))
        
        add_new_feas.append("Dir_sin")
        add_new_feas.append("Dir_cos")

        ## diff Score
        df["diffScoreBeforePlay"] = df["HomeScoreBeforePlay"] - df["VisitorScoreBeforePlay"]
        add_new_feas.append("diffScoreBeforePlay")
    
        static_features = df[df['NflId'] == df['NflIdRusher']][add_new_feas+['GameId','PlayId','X','Y','S','A','Dis','Orientation','Dir',
                                                                             'YardLine','Quarter','Down','Distance','DefendersInTheBox',
                                                                             'NflId','NflIdRusher','PossessionTeam','HomeTeamAbbr','Turf',
                                                                             'VisitorTeamAbbr','PlayDirection','GameClock','Season','Team',
                                                                             'FieldPosition']].drop_duplicates()

        return static_features


    def combine_features(relative_to_back, defense, static, deploy=deploy):
        df = pd.merge(relative_to_back,defense,on=['GameId','PlayId'],how='inner')
        df = pd.merge(df,static,on=['GameId','PlayId'],how='inner')
        
        if not deploy:
            df = pd.merge(df, outcomes, on=['GameId','PlayId'], how='inner')

        return df
    
    yardline = update_yardline(df)
    df = update_orientation(df, yardline)
    back_feats = back_features(df)
    rel_back = features_relative_to_back(df, back_feats)
    def_feats = defense_features(df)
    static_feats = static_features(df)
    basetable = combine_features(rel_back, def_feats, static_feats, deploy = deploy)
    
    return basetable

In [60]:
def create_features_02(t_):
    
    t_['fe1'] = pd.Series(np.sqrt(np.absolute(np.square(t_.X.values) - np.square(t_.Y.values))))
    t_['fe5'] = np.square(t_['S'].values) + 2 * t_['A'].values * t_['Dis'].values  # N
    t_['fe7'] = np.arccos(np.clip(t_['X'].values / t_['Y'].values, -1, 1))  # N
    t_['fe8'] = t_['S'].values / np.clip(t_['fe1'].values, 0.6, None)
    radian_angle = (90 - t_['Dir']) * np.pi / 180.0
    t_['fe10'] = np.abs(t_['S'] * np.cos(radian_angle))
    t_['fe11'] = np.abs(t_['S'] * np.sin(radian_angle))

    t_['IsRusher'] = (t_['NflId'] == t_['NflIdRusher'])
    temp = t_[t_["IsRusher"]][["Team", "PlayId"]].rename(columns={"Team":"RusherTeam"})
    t_ = t_.merge(temp, on = "PlayId")
    t_["IsRusherTeam"] = t_["Team"] == t_["RusherTeam"]    

    t_["is_left"]            = t_["PlayDirection"] == "left"
    t_["old_data"]    = t_["Season"] == 2017
    t_['YardLine_std'] = 100 - t_['YardLine']
    
    t_['Orientation_rad'] = np.mod(t_.Orientation, 360) * np.pi/180.0
    t_.loc[t_.Season >= 2018, 'Orientation_rad'] = np.mod(t_.loc[t_.Season >= 2018, 'Orientation'] - 90, 360) * np.pi/180.0
    t_['Orientation_std'] = t_.Orientation_rad
    t_.loc[t_.is_left, 'Orientation_std'] = np.mod(np.pi + t_.loc[t_.is_left, 'Orientation_rad'], 2*np.pi)
    
    t_['norm_quat'] = (t_['X']**2 + t_['Y']**2 + t_['A']**2 + t_['S']**2)
    t_['mod_quat'] = (t_['norm_quat'])**0.5
    t_['norm_X'] = t_['X'] / t_['mod_quat']
    t_['norm_Y'] = t_['Y'] / t_['mod_quat']
    t_['norm_A'] = t_['A'] / t_['mod_quat']
    t_['norm_S'] = t_['S'] / t_['mod_quat']    

    t_['seconds_need_to_first_down'] = (t_['Distance']*0.9144)/t_['Dis']
    t_['seconds_need_to_YardsLine'] = (t_['YardLine']*0.9144)/t_['Dis']    
    t_['DefendersInTheBox_vs_Distance'] = t_['DefendersInTheBox'] / t_['Distance']    

    t_['time_quarter'] = t_.GameClock.map(lambda x:transform_time_quarter(x))
    t_['time_end'] = t_.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)
    
    t_ = t_.sort_values(by = ['X']).sort_values(by = ['Dis']).sort_values(by=['PlayId', 'IsRusherTeam', 'IsRusher']).reset_index(drop = True)
    
    
    #t_.loc[t_['Season'] == 2017, 'S'] = (t_['S'][t_['Season'] == 2017] - 2.4355) / 1.2930 * 1.4551 + 2.7570

    # Novas features com base no horario do jogo
    #t_['Morning']   = t_['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >=0 and int(x[0:2]) <12) else 0)
    #t_['Afternoon'] = t_['GameClock'].apply(lambda x : 1 if (int(x[0:2]) <18 and int(x[0:2]) >=12) else 0)
    #t_['Evening']   = t_['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >= 18 and int(x[0:2]) < 24) else 0)
    #t_['GameClock'] = t_['GameClock'].apply(strtoseconds) 
    

    
    '''t_['fe12'] = np.abs(t_['A'] * np.cos(radian_angle))
    t_['fe13'] = np.abs(t_['A'] * np.sin(radian_angle))
    
    rad = 2 * np.pi * (90 - t_['Orientation']) / 360
    t_['fe14'] = np.abs(t_['S'] * np.cos(rad))
    t_['fe15'] = np.abs(t_['S'] * np.sin(rad))
    t_['fe16'] = np.abs(t_['A'] * np.cos(rad))
    t_['fe17'] = np.abs(t_['A'] * np.sin(rad))
    
        
    t_['IsRusher'] = (t_['NflId'] == t_['NflIdRusher'])
    temp = t_[t_["IsRusher"]][["Team", "PlayId"]].rename(columns={"Team":"RusherTeam"})
    t_ = t_.merge(temp, on = "PlayId")
    t_["IsRusherTeam"] = t_["Team"] == t_["RusherTeam"]    

    t_["is_left"]            = t_["PlayDirection"] == "left"
    t_["old_data"]    = t_["Season"] == 2017
    t_['YardLine_std'] = 100 - t_['YardLine']
    
    t_['Orientation_rad'] = np.mod(t_.Orientation, 360) * np.pi/180.0
    t_.loc[t_.Season >= 2018, 'Orientation_rad'] = np.mod(t_.loc[t_.Season >= 2018, 'Orientation'] - 90, 360) * np.pi/180.0
    t_['Orientation_std'] = t_.Orientation_rad
    t_.loc[t_.is_left, 'Orientation_std'] = np.mod(np.pi + t_.loc[t_.is_left, 'Orientation_rad'], 2*np.pi)
    
    t_['norm_quat'] = (t_['X']**2 + t_['Y']**2 + t_['A']**2 + t_['S']**2)
    t_['mod_quat'] = (t_['norm_quat'])**0.5
    t_['norm_X'] = t_['X'] / t_['mod_quat']
    t_['norm_Y'] = t_['Y'] / t_['mod_quat']
    t_['norm_A'] = t_['A'] / t_['mod_quat']
    t_['norm_S'] = t_['S'] / t_['mod_quat']
    
    
    Turf = {'Field Turf':'Artificial', 'A-Turf Titan':'Artificial', 'Grass':'Natural', 'UBU Sports Speed S5-M':'Artificial', 
            'Artificial':'Artificial', 'DD GrassMaster':'Artificial', 'Natural Grass':'Natural', 
            'UBU Speed Series-S5-M':'Artificial', 'FieldTurf':'Artificial', 'FieldTurf 360':'Artificial', 'Natural grass':'Natural', 'grass':'Natural', 
            'Natural':'Natural', 'Artifical':'Artificial', 'FieldTurf360':'Artificial', 'Naturall Grass':'Natural', 'Field turf':'Artificial', 
            'SISGrass':'Artificial', 'Twenty-Four/Seven Turf':'Artificial', 'natural grass':'Natural'} 

    t_['Turf'] = t_['Turf'].map(Turf)
    t_['Turf'] = t_['Turf'] == 'Natural'
    
    t_['X_std'] = t_['X']
    t_.loc[t_['is_left'], 'X_std'] = 120 - t_.loc[t_['is_left'], 'X'] 
    t_['Y_std'] = t_['Y']
    t_.loc[t_['is_left'], 'Y_std'] = 160/3 - t_.loc[t_['is_left'], 'Y'] 

    
    t_["Start"] = t_["YardLine"]
    t_['PlayDirection_new'] = t_['PlayDirection'].map({'right': 1, 'left': -1})
    t_['Orientation_new'] = 2 * np.pi * (90 - t_['Orientation']) / 360
    t_['locX'] = (t_['X'].values - t_['Start'].values) * t_['PlayDirection_new'].values
    t_['locY'] = t_['Y'].values - 53.3 / 2
    t_['velX'] = t_['S'].values * np.cos(t_['Orientation_new'].values) * t_['PlayDirection_new'].values
    t_['velY'] = t_['S'].values * np.sin(t_['Orientation_new'].values)
    t_['accX'] = t_['A'].values * np.cos(t_['Orientation_new'].values) * t_['PlayDirection_new'].values
    t_['accY'] = t_['A'].values * np.sin(t_['Orientation_new'].values)
    
    t_['YardsFromOwnGoal'] = np.where(t_.FieldPosition == t_.PossessionTeam,t_.YardLine, 50 + (50-t_.YardLine))
    t_[['prev_game', 'prev_play', 'prev_team', 'prev_yfog','prev_X','prev_Y','prev_A','prev_S']] = t_[['GameId', 'PlayId', 'Team', 'YardsFromOwnGoal','X','Y','A','S']].shift(1)

    filt = (t_.GameId==t_.prev_game) & (t_.Team==t_.prev_team) & (t_.PlayId-t_.prev_play<30)
    t_.loc[filt,'est_prev_yards'] = t_[filt]['YardsFromOwnGoal'] - t_[filt]['prev_yfog']

    t_ = t_.sort_values(by = ['X']).sort_values(by = ['Dis']).sort_values(by=['PlayId', 'IsRusherTeam', 'IsRusher']).reset_index(drop = True)
    '''
    return t_

In [63]:
%time train_basetable = create_features_01(train, False)

CPU times: user 3min 7s, sys: 6.06 s, total: 3min 13s
Wall time: 3min


In [64]:
%time train_basetable = create_features_02(train_basetable)

CPU times: user 2.18 s, sys: 228 ms, total: 2.41 s
Wall time: 1.48 s


In [65]:
# Cria uma copia do dataset para backup
X = train_basetable.copy()
X.shape
X.head()

,GameId,PlayId,back_from_scrimmage,back_oriented_down_field,back_moving_down_field,min_dist,max_dist,mean_dist,std_dist,def_min_dist,def_max_dist,def_mean_dist,def_std_dist,PlayerHeight_dense,PlayerAge,WindSpeed_dense,GameWeather_dense,Dir_sin,Dir_cos,diffScoreBeforePlay,X,Y,S,A,Dis,Orientation,Dir,YardLine,Quarter,Down,Distance,DefendersInTheBox,NflId,NflIdRusher,PossessionTeam,HomeTeamAbbr,Turf,VisitorTeamAbbr,PlayDirection,GameClock,Season,Team,FieldPosition,Yards,fe1,fe5,fe7,fe8,fe10,fe11,IsRusher,RusherTeam,IsRusherTeam,is_left,old_data,YardLine_std,Orientation_rad,Orientation_std,norm_quat,mod_quat,norm_X,norm_Y,norm_A,norm_S,seconds_need_to_first_down,seconds_need_to_YardsLine,DefendersInTheBox_vs_Distance,time_quarter,time_end
0,2017090700,20170907000118,3.75,1,0,1.449724,22.415872,8.046559,4.873845,4.593310,22.415872,9.752491,5.327299,70,25.596251,8.0,1.0,0.911690,-0.410878,0,41.25,30.53,3.63,3.35,0.38,198.02,114.26,45.0,1,3,2,6.0,2543773,2543773,NE,NE,Field Turf,KC,left,14:14:00,2017,home,NE,8,27.739531,15.7229,0.0,0.130860,3.309436,1.491487,True,home,True,True,True,55.0,3.456101,0.314508,2658.0428,51.556210,0.800098,0.592169,0.064978,0.070409,4.812632,108.284211,3.0,854,46
1,2017090700,20170907000139,4.07,0,0,0.792023,23.025872,8.614623,5.598683,4.287773,23.025872,10.297028,5.833217,70,25.596252,8.0,1.0,0.740805,0.671721,0,48.93,27.16,3.06,2.41,0.34,149.30,47.80,53.0,1,1,10,6.0,2543773,2543773,NE,NE,Field Turf,KC,left,13:52:00,2017,home,NE,3,40.699869,11.0024,0.0,0.075185,2.266862,2.055465,True,home,True,True,True,47.0,2.605777,5.747369,3146.9822,56.097970,0.872224,0.484153,0.042961,0.054547,26.894118,142.538824,0.6,832,68
2,2017090700,20170907000189,3.66,1,0,1.646390,20.726285,8.482583,4.642121,4.221670,20.726285,9.903689,5.073290,70,25.596253,8.0,1.0,0.668612,-0.743612,0,71.34,19.11,5.77,2.42,0.60,219.18,138.04,75.0,1,1,10,7.0,2543773,2543773,NE,NE,Field Turf,KC,left,13:02:00,2017,home,KC,5,68.732841,36.1969,0.0,0.083948,3.857889,4.290640,True,home,True,True,True,25.0,3.825413,0.683820,5493.7370,74.119748,0.962497,0.257826,0.032650,0.077847,15.240000,114.300000,0.7,782,118
3,2017090700,20170907000345,3.53,0,0,0.918096,9.791231,5.549379,1.983128,4.528002,9.791231,6.309354,1.834174,71,26.852933,8.0,1.0,0.995496,0.094803,0,104.47,25.36,4.45,3.20,0.46,173.78,84.56,108.0,1,2,2,9.0,2539663,2539663,NE,NE,Field Turf,KC,left,12:12:00,2017,home,KC,2,101.345209,22.7465,0.0,0.043909,4.429957,0.421875,True,home,True,True,True,-8.0,3.033033,6.174626,11587.1530,107.643639,0.970517,0.235592,0.029728,0.041340,3.975652,214.685217,4.5,732,168
4,2017090700,20170907000395,5.01,0,0,0.502892,21.214806,9.168819,5.611232,4.288088,21.214806,11.056456,5.900009,71,22.091819,8.0,1.0,0.375901,-0.926660,7,29.99,27.12,3.90,2.53,0.44,34.27,157.92,35.0,1,1,10,7.0,2557917,2557917,KC,NE,Field Turf,KC,right,12:08:00,2017,away,KC,7,12.802566,17.4364,0.0,0.304626,1.466013,3.613974,True,away,True,False,True,65.0,0.598124,0.598124,1656.5054,40.700189,0.736852,0.666336,0.062162,0.095823,20.781818,72.736364,0.7,728,172


In [80]:
best_features = ['A','S','back_from_scrimmage','back_oriented_down_field','back_moving_down_field','old_data',
                 'def_mean_dist','def_std_dist','def_min_dist','def_max_dist','min_dist','max_dist','mean_dist',
                 'PlayerAge','PlayerHeight_dense','Dis','DefendersInTheBox',
                 'Distance','Dir','Dir_sin','Dir_cos','YardLine_std','Orientation_std',
                 'WindSpeed_dense','GameWeather_dense',
                 'fe1','fe5','fe8','fe10','fe11'] #'time_quarter','time_end',
                 #'norm_quat','mod_quat','norm_X','norm_Y','norm_A','norm_S',
                 #'seconds_need_to_first_down','seconds_need_to_YardsLine',
                 #'DefendersInTheBox_vs_Distance']

## 4. Criar e avaliar alguns algoritmos de Machine Learning

### 4.1. Split Treino e Validação

In [81]:
#new_X = X.copy()
new_X = X.loc[:,best_features]
new_X.fillna(-0,inplace=True)
#new_X.replace(-np.inf,0,inplace=True)
#new_X.replace(np.inf,0,inplace=True)
new_X.head()

,A,S,back_from_scrimmage,back_oriented_down_field,back_moving_down_field,old_data,def_mean_dist,def_std_dist,def_min_dist,def_max_dist,min_dist,max_dist,mean_dist,PlayerAge,PlayerHeight_dense,Dis,DefendersInTheBox,Distance,Dir,Dir_sin,Dir_cos,YardLine_std,Orientation_std,WindSpeed_dense,GameWeather_dense,fe1,fe5,fe8,fe10,fe11
0,3.35,3.63,3.75,1,0,True,9.752491,5.327299,4.593310,22.415872,1.449724,22.415872,8.046559,25.596251,70,0.38,6.0,2,114.26,0.911690,-0.410878,55.0,0.314508,8.0,1.0,27.739531,15.7229,0.130860,3.309436,1.491487
1,2.41,3.06,4.07,0,0,True,10.297028,5.833217,4.287773,23.025872,0.792023,23.025872,8.614623,25.596252,70,0.34,6.0,10,47.80,0.740805,0.671721,47.0,5.747369,8.0,1.0,40.699869,11.0024,0.075185,2.266862,2.055465
2,2.42,5.77,3.66,1,0,True,9.903689,5.073290,4.221670,20.726285,1.646390,20.726285,8.482583,25.596253,70,0.60,7.0,10,138.04,0.668612,-0.743612,25.0,0.683820,8.0,1.0,68.732841,36.1969,0.083948,3.857889,4.290640
3,3.20,4.45,3.53,0,0,True,6.309354,1.834174,4.528002,9.791231,0.918096,9.791231,5.549379,26.852933,71,0.46,9.0,2,84.56,0.995496,0.094803,-8.0,6.174626,8.0,1.0,101.345209,22.7465,0.043909,4.429957,0.421875
4,2.53,3.90,5.01,0,0,True,11.056456,5.900009,4.288088,21.214806,0.502892,21.214806,9.168819,22.091819,71,0.44,7.0,10,157.92,0.375901,-0.926660,65.0,0.598124,8.0,1.0,12.802566,17.4364,0.304626,1.466013,3.613974


In [82]:
# Criar um dataset somente com as colunas mais importantes conforme visto anteriormente
target = X.Yards

y = np.zeros((target.shape[0], 199))
for idx, target in enumerate(list(target)):
    y[idx][99 + target] = 1
    
# Normalizando as variaveis do dataset de treino
scaler = StandardScaler()
new_X = scaler.fit_transform(new_X)
new_X.shape, y.shape

((23171, 30), (23171, 199))

### 4.2. Teste 3 modelos (LightGBM, RF e NN)

In [83]:
# Calculate CRPS score
def crps_score(y_prediction, y_valid, shape=X.shape[0]):
    y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_prediction, axis=1), 0, 1)
    val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * shape)
    crps = np.round(val_s, 6)
    
    return crps

In [84]:
class CRPSCallback(Callback):
    
    def __init__(self,validation, predict_batch_size=20, include_on_batch=False):
        super(CRPSCallback, self).__init__()
        self.validation = validation
        self.predict_batch_size = predict_batch_size
        self.include_on_batch = include_on_batch
        
        print('validation shape',len(self.validation))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_train_begin(self, logs={}):
        if not ('CRPS_score_val' in self.params['metrics']):
            self.params['metrics'].append('CRPS_score_val')

    def on_batch_end(self, batch, logs={}):
        if (self.include_on_batch):
            logs['CRPS_score_val'] = float('-inf')

    def on_epoch_end(self, epoch, logs={}):
        logs['CRPS_score_val'] = float('-inf')
            
        if (self.validation):
            X_valid, y_valid = self.validation[0], self.validation[1]
            y_pred = self.model.predict(X_valid)
            y_true = np.clip(np.cumsum(y_valid, axis=1), 0, 1)
            y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
            val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * X_valid.shape[0])
            val_s = np.round(val_s, 6)
            logs['CRPS_score_val'] = val_s

In [85]:
def get_nn(x_tr,y_tr,x_val,y_val,shape):
    #K.clear_session()
    inp = Input(shape = (x_tr.shape[1],))
    
    x = Dense(1024, input_dim=X.shape[1], activation='relu')(inp)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    
    out = Dense(199, activation='softmax')(x)
    model = Model(inp,out)
    
    model.compile(optimizer = 'Adam', #optimizers.adam(lr = 0.005, decay = 1e-06),
                  loss='categorical_crossentropy', 
                  metrics=[])
     
    es = EarlyStopping(monitor='CRPS_score_val', 
                       mode='min',
                       restore_best_weights=True, 
                       verbose=1, 
                       patience=10)

    mc = ModelCheckpoint('best_model.h5',monitor='CRPS_score_val',mode='min',save_best_only=True, 
                         verbose=1, save_weights_only=True)
    
    bsz = 1024
    steps = x_tr.shape[0]/bsz
    
    model.fit(x_tr, y_tr,
              callbacks=[CRPSCallback(validation = (x_val,y_val)),es,mc], 
              epochs=200, 
              #steps_per_epoch = steps,
              batch_size=bsz,
              verbose=1)
    
    model.load_weights("best_model.h5")
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    crps = crps_score(y_pred, y_valid, shape=shape)

    return model,crps

In [86]:
# increase the n_estimators (50 -> 64) to see if this improves CV
def get_rf(x_tr, y_tr, x_val, y_val, shape):
    model = RandomForestRegressor(bootstrap=False, max_features=0.3, min_samples_leaf=15, 
                                  min_samples_split=8, n_estimators=100, n_jobs=-1, random_state=42)
    model.fit(x_tr, y_tr)
    
    y_pred = model.predict(x_val)
    y_valid = y_val
    crps = crps_score(y_pred, y_valid, shape=shape)
    
    return model, crps

In [100]:
%%time

loop = 2
fold = 10

oof_nn  = np.zeros([loop, y.shape[0], y.shape[1]])
oof_rf  = np.zeros([loop, y.shape[0], y.shape[1]])

models_nn = []
crps_csv_nn = []
models_rf = []
crps_csv_rf = []

for k in range(loop):
    kfold = KFold(fold, random_state = 42 + k, shuffle = True)
    for k_fold, (tr_inds, val_inds) in enumerate(kfold.split(y)):
        print("-----------")
        print(f'Loop {k+1}/{loop}' + f' Fold {k_fold+1}/{fold}')
        print("-----------")
        
        tr_x, tr_y = new_X[tr_inds], y[tr_inds]
        val_x, val_y = new_X[val_inds], y[val_inds]
        
        # Train NN
        nn, crps_nn = get_nn(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0])
        models_nn.append(nn)
        print("the %d fold crps (NN) is %f"%((k_fold+1), crps_nn))
        crps_csv_nn.append(crps_nn)
        
        # Train RF
        rf, crps_rf = get_rf(tr_x, tr_y, val_x, val_y, shape=val_x.shape[0])
        models_rf.append(rf)
        print("the %d fold crps (RF) is %f"%((k_fold+1), crps_rf))
        crps_csv_rf.append(crps_rf)
        
        #Predict OOF
        oof_nn[k, val_inds, :] = nn.predict(val_x)
        oof_rf[k, val_inds, :] = rf.predict(val_x)

-----------
Loop 1/2 Fold 1/10
-----------
validation shape 2
Epoch 1/200
20853/20853 [==============================] - 2s 107us/step - loss: 5.5132

Epoch 00001: CRPS_score_val improved from inf to 0.08210, saving model to best_model.h5
Epoch 2/200
20853/20853 [==============================] - 1s 68us/step - loss: 4.9808

Epoch 00002: CRPS_score_val improved from 0.08210 to 0.07480, saving model to best_model.h5
Epoch 3/200
20853/20853 [==============================] - 1s 67us/step - loss: 4.4987

Epoch 00003: CRPS_score_val improved from 0.07480 to 0.05570, saving model to best_model.h5
Epoch 4/200
20853/20853 [==============================] - 1s 69us/step - loss: 4.0524

Epoch 00004: CRPS_score_val improved from 0.05570 to 0.03328, saving model to best_model.h5
Epoch 5/200
20853/20853 [==============================] - 1s 69us/step - loss: 3.6741

Epoch 00005: CRPS_score_val improved from 0.03328 to 0.02089, saving model to best_model.h5
Epoch 6/200
20853/20853 [================

20853/20853 [==============================] - 2s 80us/step - loss: 2.6068

Epoch 00049: CRPS_score_val did not improve from 0.01273
Epoch 50/200
20853/20853 [==============================] - 2s 74us/step - loss: 2.6070

Epoch 00050: CRPS_score_val did not improve from 0.01273
Epoch 51/200
20853/20853 [==============================] - 2s 78us/step - loss: 2.6085

Epoch 00051: CRPS_score_val did not improve from 0.01273
Epoch 52/200
20853/20853 [==============================] - 2s 80us/step - loss: 2.6054

Epoch 00052: CRPS_score_val did not improve from 0.01273
Epoch 53/200
20853/20853 [==============================] - 2s 77us/step - loss: 2.6008

Epoch 00053: CRPS_score_val did not improve from 0.01273
Epoch 54/200
20853/20853 [==============================] - 2s 76us/step - loss: 2.5995

Epoch 00054: CRPS_score_val did not improve from 0.01273
Epoch 55/200
20853/20853 [==============================] - 2s 77us/step - loss: 2.5974

Epoch 00055: CRPS_score_val did not improve from

20854/20854 [==============================] - 1s 63us/step - loss: 2.6749

Epoch 00029: CRPS_score_val improved from 0.01272 to 0.01271, saving model to best_model.h5
Epoch 30/200
20854/20854 [==============================] - 1s 63us/step - loss: 2.6701

Epoch 00030: CRPS_score_val did not improve from 0.01271
Epoch 31/200
20854/20854 [==============================] - 1s 64us/step - loss: 2.6684

Epoch 00031: CRPS_score_val did not improve from 0.01271
Epoch 32/200
20854/20854 [==============================] - 1s 66us/step - loss: 2.6660

Epoch 00032: CRPS_score_val improved from 0.01271 to 0.01270, saving model to best_model.h5
Epoch 33/200
20854/20854 [==============================] - 1s 65us/step - loss: 2.6589

Epoch 00033: CRPS_score_val did not improve from 0.01270
Epoch 34/200
20854/20854 [==============================] - 1s 63us/step - loss: 2.6614

Epoch 00034: CRPS_score_val did not improve from 0.01270
Epoch 35/200
20854/20854 [==============================] - 1s 63us

Epoch 26/200
20854/20854 [==============================] - 1s 65us/step - loss: 2.6767

Epoch 00026: CRPS_score_val improved from 0.01260 to 0.01260, saving model to best_model.h5
Epoch 27/200
20854/20854 [==============================] - 1s 70us/step - loss: 2.6703

Epoch 00027: CRPS_score_val improved from 0.01260 to 0.01259, saving model to best_model.h5
Epoch 28/200
20854/20854 [==============================] - 1s 67us/step - loss: 2.6743

Epoch 00028: CRPS_score_val improved from 0.01259 to 0.01257, saving model to best_model.h5
Epoch 29/200
20854/20854 [==============================] - 1s 68us/step - loss: 2.6745

Epoch 00029: CRPS_score_val improved from 0.01257 to 0.01255, saving model to best_model.h5
Epoch 30/200
20854/20854 [==============================] - 2s 72us/step - loss: 2.6681

Epoch 00030: CRPS_score_val improved from 0.01255 to 0.01254, saving model to best_model.h5
Epoch 31/200
20854/20854 [==============================] - 1s 65us/step - loss: 2.6558

Epoch 

20854/20854 [==============================] - 1s 70us/step - loss: 2.8525

Epoch 00010: CRPS_score_val improved from 0.01355 to 0.01341, saving model to best_model.h5
Epoch 11/200
20854/20854 [==============================] - 1s 71us/step - loss: 2.8212

Epoch 00011: CRPS_score_val improved from 0.01341 to 0.01335, saving model to best_model.h5
Epoch 12/200
20854/20854 [==============================] - 2s 74us/step - loss: 2.7877

Epoch 00012: CRPS_score_val improved from 0.01335 to 0.01329, saving model to best_model.h5
Epoch 13/200
20854/20854 [==============================] - 2s 74us/step - loss: 2.7736

Epoch 00013: CRPS_score_val improved from 0.01329 to 0.01320, saving model to best_model.h5
Epoch 14/200
20854/20854 [==============================] - 1s 67us/step - loss: 2.7557

Epoch 00014: CRPS_score_val improved from 0.01320 to 0.01314, saving model to best_model.h5
Epoch 15/200
20854/20854 [==============================] - 1s 67us/step - loss: 2.7466

Epoch 00015: CRPS_s

...

20853/20853 [==============================] - 1s 72us/step - loss: 2.7595

Epoch 00015: CRPS_score_val improved from 0.01263 to 0.01259, saving model to best_model.h5
Epoch 16/200
20853/20853 [==============================] - 1s 71us/step - loss: 2.7385

Epoch 00016: CRPS_score_val improved from 0.01259 to 0.01253, saving model to best_model.h5
Epoch 17/200
20853/20853 [==============================] - 2s 72us/step - loss: 2.7256

Epoch 00017: CRPS_score_val improved from 0.01253 to 0.01250, saving model to best_model.h5
Epoch 18/200
20853/20853 [==============================] - 2s 74us/step - loss: 2.7251

Epoch 00018: CRPS_score_val improved from 0.01250 to 0.01249, saving model to best_model.h5
Epoch 19/200
20853/20853 [==============================] - 1s 72us/step - loss: 2.7201

Epoch 00019: CRPS_score_val improved from 0.01249 to 0.01246, saving model to best_model.h5
Epoch 20/200
20853/20853 [==============================] - 1s 70us/step - loss: 2.7089

Epoch 00020: CRPS_s

20853/20853 [==============================] - 1s 71us/step - loss: 2.5711

Epoch 00065: CRPS_score_val did not improve from 0.01223
Epoch 66/200
20853/20853 [==============================] - 2s 73us/step - loss: 2.5673

Epoch 00066: CRPS_score_val did not improve from 0.01223
Epoch 67/200
20853/20853 [==============================] - 2s 73us/step - loss: 2.5721
Restoring model weights from the end of the best epoch

Epoch 00067: CRPS_score_val did not improve from 0.01223
Epoch 00067: early stopping
the 1 fold crps (NN) is 0.012235
the 1 fold crps (RF) is 0.012438
-----------
Loop 2/2 Fold 2/10
-----------
validation shape 2
Epoch 1/200
20854/20854 [==============================] - 2s 108us/step - loss: 5.5395

Epoch 00001: CRPS_score_val improved from inf to 0.08147, saving model to best_model.h5
Epoch 2/200
20854/20854 [==============================] - 1s 68us/step - loss: 4.9794

Epoch 00002: CRPS_score_val improved from 0.08147 to 0.07530, saving model to best_model.h5
Epoch 3

20854/20854 [==============================] - 1s 70us/step - loss: 2.6231
Restoring model weights from the end of the best epoch

Epoch 00045: CRPS_score_val did not improve from 0.01235
Epoch 00045: early stopping
the 2 fold crps (NN) is 0.012347
the 2 fold crps (RF) is 0.012511
-----------
Loop 2/2 Fold 3/10
-----------
validation shape 2
Epoch 1/200
20854/20854 [==============================] - 2s 110us/step - loss: 5.5433

Epoch 00001: CRPS_score_val improved from inf to 0.08200, saving model to best_model.h5
Epoch 2/200
20854/20854 [==============================] - 1s 70us/step - loss: 5.0035

Epoch 00002: CRPS_score_val improved from 0.08200 to 0.07506, saving model to best_model.h5
Epoch 3/200
20854/20854 [==============================] - 1s 69us/step - loss: 4.5028

Epoch 00003: CRPS_score_val improved from 0.07506 to 0.05468, saving model to best_model.h5
Epoch 4/200
20854/20854 [==============================] - 1s 65us/step - loss: 4.0422

Epoch 00004: CRPS_score_val imp

20854/20854 [==============================] - 1s 71us/step - loss: 2.6164

Epoch 00047: CRPS_score_val did not improve from 0.01308
Epoch 48/200
20854/20854 [==============================] - 2s 74us/step - loss: 2.6144

Epoch 00048: CRPS_score_val did not improve from 0.01308
Epoch 49/200
20854/20854 [==============================] - 2s 73us/step - loss: 2.6053

Epoch 00049: CRPS_score_val improved from 0.01308 to 0.01308, saving model to best_model.h5
Epoch 50/200
20854/20854 [==============================] - 2s 74us/step - loss: 2.6089

Epoch 00050: CRPS_score_val improved from 0.01308 to 0.01307, saving model to best_model.h5
Epoch 51/200
20854/20854 [==============================] - 2s 72us/step - loss: 2.5988

Epoch 00051: CRPS_score_val did not improve from 0.01307
Epoch 52/200
20854/20854 [==============================] - 2s 73us/step - loss: 2.6016

Epoch 00052: CRPS_score_val did not improve from 0.01307
Epoch 53/200
20854/20854 [==============================] - 2s 72us

KeyboardInterrupt: 

In [89]:
crps_oof_nn  = []
crps_oof_rf  = []

for k in range(loop):
    crps_oof_nn.append(crps_score(oof_nn[k,...], y))
    crps_oof_rf.append(crps_score(oof_rf[k,...], y))

In [90]:
print("mean crps (NN) is %f"%np.mean(crps_csv_nn))
print("mean crps (RF) is %f"%np.mean(crps_csv_rf))

mean crps (NN) is 0.012626
mean crps (RF) is 0.012821


In [91]:
print("mean OOF crps (NN) is %f"%np.mean(crps_oof_nn))
print("mean OOF crps (RF) is %f"%np.mean(crps_oof_rf))

mean OOF crps (NN) is 0.012626
mean OOF crps (RF) is 0.012821


## Blending Weight Optimisation

In [92]:
def weight_opt(oof_nn, oof_rf, y_true):
    weight_nn = np.inf
    best_crps = np.inf
    
    for i in np.arange(0, 1.01, 0.05):
        crps_blend = np.zeros(oof_nn.shape[0])
        for k in range(oof_nn.shape[0]):
            crps_blend[k] = crps_score(i * oof_nn[k,...] + (1-i) * oof_rf[k,...], y_true)
        if np.mean(crps_blend) < best_crps:
            best_crps = np.mean(crps_blend)
            weight_nn = round(i, 2)
            
        print(str(round(i, 2)) + ' : mean crps (Blend) is ', round(np.mean(crps_blend), 6))
        
    print('-'*36)
    print('Best weight for NN: ', weight_nn)
    print('Best weight for RF: ', round(1-weight_nn, 2))
    print('Best mean crps (Blend): ', round(best_crps, 6))
    
    return weight_nn, round(1-weight_nn, 2)

In [93]:
weight_nn, weight_rf = weight_opt(oof_nn, oof_rf, y)

0.0 : mean crps (Blend) is  0.012821
0.05 : mean crps (Blend) is  0.012794
0.1 : mean crps (Blend) is  0.01277
0.15 : mean crps (Blend) is  0.012746
0.2 : mean crps (Blend) is  0.012725
0.25 : mean crps (Blend) is  0.012705
0.3 : mean crps (Blend) is  0.012688
0.35 : mean crps (Blend) is  0.012671
0.4 : mean crps (Blend) is  0.012657
0.45 : mean crps (Blend) is  0.012644
0.5 : mean crps (Blend) is  0.012634
0.55 : mean crps (Blend) is  0.012625
0.6 : mean crps (Blend) is  0.012618
0.65 : mean crps (Blend) is  0.012612
0.7 : mean crps (Blend) is  0.012609
0.75 : mean crps (Blend) is  0.012607
0.8 : mean crps (Blend) is  0.012607
0.85 : mean crps (Blend) is  0.012609
0.9 : mean crps (Blend) is  0.012612
0.95 : mean crps (Blend) is  0.012618
1.0 : mean crps (Blend) is  0.012626
------------------------------------
Best weight for NN:  0.75
Best weight for RF:  0.25
Best mean crps (Blend):  0.012607


In [ ]:
1. Best mean crps (Blend):  0.012616
2. Best mean crps (Blend):  0.012621
3. Best mean crps (Blend):  0.012607

In [ ]:
def predict(x_te, models_nn, models_rf, weight_nn, weight_rf):
    model_num_nn = len(models_nn)
    model_num_rf = len(models_rf)
    for k,m in enumerate(models_nn):
        if k==0:
            y_pred_nn = m.predict(x_te, batch_size=1024)
            y_pred_rf = models_rf[k].predict(x_te)
        else:
            y_pred_nn += m.predict(x_te, batch_size=1024)
            y_pred_rf += models_rf[k].predict(x_te)
            
    y_pred_nn = y_pred_nn / model_num_nn
    y_pred_rf = y_pred_rf / model_num_rf
    
    return weight_nn * y_pred_nn + weight_rf * y_pred_rf

## 5. Realizar a submissão para o Kaggle

In [ ]:
%%time

if  TRAIN_OFFLINE==False:
    
    from kaggle.competitions import nflrush
    env = nflrush.make_env()
    iter_test = env.iter_test()
    df_prev = pd.DataFrame()
    df_test = pd.DataFrame()

    for (test_df, sample_prediction_df) in tqdm_notebook(iter_test):
        basetable = create_features_01(test_df, deploy=True)
        basetable = create_features_02(basetable)

        # Considerar somente as colunas do Feature Selection
        basetable = basetable.loc[:,best_features]
        basetable.fillna(0,inplace=True)
    
        scaled_basetable = scaler.transform(basetable)

        #y_pred = predict(scaled_basetable)
        y_pred = predict(scaled_basetable, models_nn, models_rf, weight_nn, weight_rf)
        y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1).tolist()[0]

        preds_df = pd.DataFrame(data=[y_pred], columns=sample_prediction_df.columns)
        
        df_test = df_test.append(basetable)
        df_prev = df_prev.append(preds_df)
    
        env.predict(preds_df)

    env.write_submission_file()

In [ ]:
df_test.head()

In [ ]:
df_prev.head()